# DeepAgents and Human-in-the-Loop

## Types of Interrupts

The three right now are **allow_accept**, **allow_edit**, and **allow_respond**.

- ```allow_accept``` adds an interrupt and allows the human user to accept that tool call and it will just continue from there.
- ```allow_edit``` allows the human user to edit the tool call and maybe even the tool name that has been called and then continue with those edited arguments.
- ```allow_respond``` instead responds to the tool call and places the user's response in the tool message. So it doesn't actually execute any tool call. It rather mocks that out with the response and then sends that back to the model.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os

from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:
from deepagents import create_deep_agent
from langchain_core.tools import tool

In [4]:
@tool
def send_email(content: str, recepient:str):
    """Send an email to the recepient with the content"""
    print(f"Sending email to {recepient} with content: {content}")
    return "Email sent successfully"

In [5]:
hitl_config = {"send_email": True}

This is equivalent to:

```python
hitl_config = {
    "send_email": {
        "allow_accept": True,
        "allow_edit": True,
        "allow_respond": True,
        "description": "⚠️ Operation requires approval",
    }
}
```

In [6]:
prompt = "You are a helpful email assistant."
agent = create_deep_agent([send_email], prompt, interrupt_config=hitl_config)

In [7]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command

checkpointer = InMemorySaver()
agent.checkpointer = checkpointer

In [8]:
message = "Tell jim@example.com ill be late."

# Accept

In [9]:
config = {"configurable": {"thread_id": "1-accept"}}
for s in agent.stream({"messages": [{"role": "user", "content": message}]}, config=config):
    print(s)

{'agent': {'messages': [AIMessage(content=[{'text': "I'll send an email to Jim letting him know you'll be late.", 'type': 'text'}, {'id': 'toolu_01Msw9TKXWxytoGD41NQCWBr', 'input': {'recepient': 'jim@example.com', 'content': "Hi Jim,\n\nI wanted to let you know that I'll be running late today.\n\nThanks for your understanding.\n\nBest regards"}, 'name': 'send_email', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_019CvUDPYk25FS5Qfu6spEDG', 'model': 'claude-sonnet-4-20250514', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 4928, 'output_tokens': 118, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-20250514'}, id='run--c09778e1-d27c-4e09-8566-d036235d3a50-0', tool_calls=[{'name': 'send_email', 'args': {'recepient': 'jim@example.com', 'content': "Hi Ji

## Command - type = accept 

To continue from here, I can use the ```Command``` object. In this case, I want to pass back some **response** to the human interrupt. So, I'm passing back this ```Command``` object. I'm going to pass back this ```resume``` keyword argument and I'm going to pass in a list of objects.

Why am I passing in a list? 

So, this interrupt, the value is a list itself: ```(Interrupt(value=[```. So, you can actually interrupt multiple times. That's actually ***disabled*** for deep agents, but in LangGraph in general, you can interact multiple times. And so that's why it's a list. The one argument I'm going to pass in is just a **dictionary**, and it's going to be ```type``` equals ```accept```. There's no other arguments needed because accept is so simple. 

If we resume from there, we can see that it goes through this ```post_model_hook```. **That's where the interrupt is happening**.

In [10]:
for s in agent.stream(Command(resume=[{"type": "accept"}]), config=config):
    print(s)

{'post_model_hook': {'messages': [AIMessage(content=[{'text': "I'll send an email to Jim letting him know you'll be late.", 'type': 'text'}, {'id': 'toolu_01Msw9TKXWxytoGD41NQCWBr', 'input': {'recepient': 'jim@example.com', 'content': "Hi Jim,\n\nI wanted to let you know that I'll be running late today.\n\nThanks for your understanding.\n\nBest regards"}, 'name': 'send_email', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_019CvUDPYk25FS5Qfu6spEDG', 'model': 'claude-sonnet-4-20250514', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 4928, 'output_tokens': 118, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-20250514'}, id='run--c09778e1-d27c-4e09-8566-d036235d3a50-0', tool_calls=[{'name': 'send_email', 'args': {'recepient': 'jim@example.com', 'conten

# Edit

In [11]:
config = {"configurable": {"thread_id": "2-edit"}}
for s in agent.stream({"messages": [{"role": "user", "content": message}]}, config=config):
    print(s)

{'agent': {'messages': [AIMessage(content=[{'text': "I'll send an email to Jim letting him know you'll be late.", 'type': 'text'}, {'id': 'toolu_011jQp6VCLjVEiPeMCCsrFkq', 'input': {'recepient': 'jim@example.com', 'content': "Hi Jim,\n\nI wanted to let you know that I'll be running late today.\n\nThanks for understanding.\n\nBest regards"}, 'name': 'send_email', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01YYph2Bn6mvS7EUUTFMoPQg', 'model': 'claude-sonnet-4-20250514', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 4928, 'output_tokens': 117, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-20250514'}, id='run--5b31d0e3-5477-4d61-be88-b3ec940e7421-0', tool_calls=[{'name': 'send_email', 'args': {'recepient': 'jim@example.com', 'content': "Hi Jim,\n\

## Command - type = edit

I'm still going to use ```Command```. I'm still going to use ```resume```. But now the ```type``` is going to be ```edit```. And I'm going to pass in another thing ```args```.

What's in args?

So args is now a **dictionary** representing what the tool to execute instead should be. So I can see it has an ```action``` name
and this will be ```send_email``` because this is the name of the tool that I have. And then the ```args``` here will be the arguments
to that tool. And so I am now overwriting this with ```content``` field I'll be late. So this is now going to be the new input to the email. And I am overwriting this with ```recepient``` field jim@gmail.com, to be the new recepient. 

In [12]:
args = {"action": "send_email", "args": {"content": "sorry i will be late!", "recepient": "jim@gmail.com"}}
for s in agent.stream(Command(resume=[{"type": "edit", "args": args}]), config=config):
    print(s)

{'post_model_hook': {'messages': [AIMessage(content=[{'text': "I'll send an email to Jim letting him know you'll be late.", 'type': 'text'}, {'id': 'toolu_011jQp6VCLjVEiPeMCCsrFkq', 'input': {'recepient': 'jim@example.com', 'content': "Hi Jim,\n\nI wanted to let you know that I'll be running late today.\n\nThanks for understanding.\n\nBest regards"}, 'name': 'send_email', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01YYph2Bn6mvS7EUUTFMoPQg', 'model': 'claude-sonnet-4-20250514', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 4928, 'output_tokens': 117, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-20250514'}, id='run--5b31d0e3-5477-4d61-be88-b3ec940e7421-0', tool_calls=[{'type': 'tool_call', 'name': 'send_email', 'args': {'content': 'sorry i wil

# Response

In [13]:
config = {"configurable": {"thread_id": "3-response"}}
for s in agent.stream({"messages": [{"role": "user", "content": message}]}, config=config):
    print(s)

{'agent': {'messages': [AIMessage(content=[{'text': "I'll send an email to Jim letting him know you'll be late.", 'type': 'text'}, {'id': 'toolu_0128NS4tNf6u4RaQjhQdKbNE', 'input': {'recepient': 'jim@example.com', 'content': "Hi Jim,\n\nI wanted to let you know that I'll be running late today.\n\nThanks for understanding.\n\nBest regards"}, 'name': 'send_email', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01FVtJH8DWZeLVfm36zmHdDM', 'model': 'claude-sonnet-4-20250514', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 4928, 'output_tokens': 117, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-20250514'}, id='run--8920200e-3c07-4c4d-b71c-4d78a10014d6-0', tool_calls=[{'name': 'send_email', 'args': {'recepient': 'jim@example.com', 'content': "Hi Jim,\n\

## Command - type = response

So now the ```type``` is different, it's ```response```. And the arguments, they exist, but they're just a string. And so this string is going to be used as the **tool call message** instead of actually ***running the tool***. So just to repeat that, it's not going to call the tool when you pass back response. Rather, it's going to send this as the tool call ```message```. And so here we can see that I'm passing an ERROR. 

User interrupted with feedback. Sign it from Harrison.

So basically what this is going to do is, this is going to tell the LLM that there was an ERROR calling the tool and that the user interrupted and it wants it to be signed as Harrison. And this is all enabled by human loop.

This ```post_model_hook``` adds a message and it adds a tool call with the same tool call ID as before with the ```content``` error user
interrupted with feedback. It now goes to the agent node. So it doesn't go to the tool node because there's no tool calls because
we **mocked out** that tool call response. So now we go back to this agent node and we can see that it regenerates a call to an LLM and now it signs it as Harrison.

In [14]:
hitl_response = """ERROR: user interrupted with feedback: 

Sign it from Harrison"""

for s in agent.stream(Command(resume=[{"type": "response", "args": hitl_response}]), config=config):
    print(s)

{'post_model_hook': {'messages': [{'type': 'tool', 'tool_call_id': 'toolu_0128NS4tNf6u4RaQjhQdKbNE', 'content': 'ERROR: user interrupted with feedback: \n\nSign it from Harrison'}]}}
{'agent': {'messages': [AIMessage(content=[{'text': 'Let me send that email again with your signature:', 'type': 'text'}, {'id': 'toolu_01RKdxzhzX5G59ndzp48woqc', 'input': {'recepient': 'jim@example.com', 'content': "Hi Jim,\n\nI wanted to let you know that I'll be running late today.\n\nThanks for understanding.\n\nBest regards,\nHarrison"}, 'name': 'send_email', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01NpwQPtGwUEurvdXBvkZDns', 'model': 'claude-sonnet-4-20250514', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 5070, 'output_tokens': 115, 'server_tool_use': None, 'service_tier': 'standard'}, 'model

## Interrupt again

Then it interrupts again because it goes back to this human in the loop config. There's now this new email that needs to be sent. And now I can accept it. 

In [15]:
for s in agent.stream(Command(resume=[{"type": "accept"}]), config=config):
    print(s)

{'post_model_hook': {'messages': [AIMessage(content=[{'text': 'Let me send that email again with your signature:', 'type': 'text'}, {'id': 'toolu_01RKdxzhzX5G59ndzp48woqc', 'input': {'recepient': 'jim@example.com', 'content': "Hi Jim,\n\nI wanted to let you know that I'll be running late today.\n\nThanks for understanding.\n\nBest regards,\nHarrison"}, 'name': 'send_email', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01NpwQPtGwUEurvdXBvkZDns', 'model': 'claude-sonnet-4-20250514', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 5070, 'output_tokens': 115, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-20250514'}, id='run--40d1a1f0-c93c-439c-96b0-f2174211ff06-0', tool_calls=[{'name': 'send_email', 'args': {'recepient': 'jim@example.com', 'content':